In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load



# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [5]:
pip install xgboost

     |████████████████████████████████| 297.1 MB 30 kB/s s eta 0:00:01    |████████▎                       | 76.8 MB 14.3 MB/s eta 0:00:16     |██████████████████████▉         | 212.4 MB 7.6 MB/s eta 0:00:12
     |████████████████████████████████| 16.8 MB 16.3 MB/s eta 0:00:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
daal4py 2021.5.0 requires daal==2021.4.0, which is not installed.
tensorflow 2.15.0.post1 requires numpy<2.0.0,>=1.23.5, but you have numpy 1.22.4 which is incompatible.
numba 0.55.1 requires numpy<1.22,>=1.18, but you have numpy 1.22.4 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [4]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from wordcloud import WordCloud

import re
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import f1_score, confusion_matrix

/home/fernando/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


ModuleNotFoundError: No module named 'xgboost'

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('./nlp-getting-started/train.csv')

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.isnull().sum()

In [ ]:
df.head()

In [ ]:
df.drop(columns=['id', 'keyword', 'location'],inplace=True)

## Data Processing

In [ ]:
df.head()

In [ ]:
def clean_text(text):
    
    text = text.lower()
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remove special characters, quotes, numbers, and other non-text characters
    text = re.sub(r'\s+', ' ', text) # Replace multiple whitespace characters with a single space   
    text = text.strip()  # Remove leading and trailing whitespace
    
    # removing stopword from text
    stopword = stopwords.words('english')
    text = [word for word in text.split() if word not in stopword ]
    
    # stemming text get word in it's root form
    sb_stem = SnowballStemmer('english')
    text = ' '.join([sb_stem.stem(word) for word in text ])
    
    return text


In [ ]:
df['text'] = df['text'].apply(clean_text)

In [ ]:
df['text_length'] = df['text'].apply(len)

# EDA

In [ ]:
df.sample()

In [ ]:
target = df['target'].value_counts()

In [ ]:
sns.barplot(x=target.index,y=target.values)

In [ ]:
sns.boxenplot(data=df,x='target',y='text_length')    

In [ ]:
px.histogram(data_frame=df, x='text_length',nbins=50,color='target', width=800, height=400)

In [ ]:
avg_len = df.groupby('target')['text_length'].mean()

In [ ]:
px.bar(x=avg_len.index,y= avg_len.values, width=500, height=400,color=avg_len.index)

In [ ]:
plt.figure(figsize=(15,10))
plot_num = 1
for i in df['target'].unique():
    plt.subplot(3,2,plot_num)
    wordcloud = WordCloud(height=400,width=800,background_color='white').generate(' '.join(df[df['target']==i]['text']))
    plt.title(f'word cloud for {i}')
    plt.imshow(wordcloud)
    plt.tight_layout()
    plot_num += 1
plt.show()

In [ ]:
df.head()

## Vectorize

In [ ]:
X = df['text']
y = df['target']

In [ ]:
tfid_vect = TfidfVectorizer(max_features=5000)

In [ ]:
X_vectorize = tfid_vect.fit_transform(X)

In [ ]:
tfid_vect.get_feature_names_out()

## Create Train and Validation Data

In [ ]:
X_train,X_val,y_train,y_val = train_test_split(X_vectorize,y,test_size=0.2,random_state=42)

In [ ]:
X_train.shape

In [ ]:
X_val.shape

## Training Model

In [ ]:
classifier = {
             'Logistic Regression' : LogisticRegression(),
             'SVC' : SVC(),
             'BernoulliNB' : BernoulliNB(),
             'MultinomialNB': MultinomialNB(),
             'RandomForestClassifier' : RandomForestClassifier(),
             'XGBClassifier' : XGBClassifier()
             }

In [ ]:
for name, model in classifier.items():
    model = model
    model.fit(X_train,y_train)
    y_predict = model.predict(X_val)
    f1score = f1_score(y_val,y_predict)
    confusion_mat = confusion_matrix(y_val,y_predict)
    print('classifier : ', name)
    print('f1-score   : ',f1score)
    print('confusion matrix : \n', confusion_mat)
    fig = px.imshow(confusion_mat,text_auto=True)
    fig.update_layout(height=300, width=400, xaxis_visible=False, yaxis_visible=False)
    fig.show()
    print("="*50)

#### From the above, the BernoulliNB model has the best F1 score.

In [ ]:
bernoulli = BernoulliNB()

In [ ]:
bernoulli.fit(X_train,y_train)

In [ ]:
test_df = pd.read_csv('/kaggle/input/nlp-getting-started/test.csv')

In [ ]:
X_test = tfid_vect.transform(test_df['text'])

In [ ]:
test_df['target'] = bernoulli.predict(X_test)

In [ ]:
test_df[['id','target']].to_csv('submission.csv',index=False)